In [8]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [9]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [1]:
#ps.head()

In [10]:
ps = ps[["Date", "User", "Post patch?", "Post patch pipette R", "Nucleus sucked in?", "Notes","Internal version", "PCR cycles", "SM_QC_PF", "Resolution index", "Bad dates"]]
#ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps.dropna(subset = ["Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["Bad dates"] != 'x']
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["PCR cycles"] == 21]
ps = ps.drop('Bad dates', 1)
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
#ps["PCR cycles"].value_counts()
ps

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4719,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4722,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4723,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4724,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4727,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4728,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000


In [119]:
total = ps["Post patch?"].count()
total
#ps['Post patch?'].value_counts()

1706

In [120]:
nuc = ps[(ps["Post patch?"] == "Nucleated") | (ps["Post patch?"] == "Partial-Nucleus") | (ps["Post patch?"] == "nucleus_present") | (ps["Post patch?"] == "nucleus_visible")]
nuc

#dic = {'Nucleated': 'test',
    #  'nucelated': 'Nucleated'
    # }

#ps['Post patch?'] = ps['Post patch?'].map(lambda x: dic[x])

,Date,User,Post patch?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4720,2017-04-19,P2,Nucleated,NaN,AiV2,21.0,pass,0.000000
4722,2017-04-19,P2,Nucleated,NaN,AiV2,21.0,pass,0.322631
4724,2017-04-19,P2,Nucleated,NaN,AiV2,21.0,pass,NaN
4737,2017-04-20,P1,Nucleated,NaN,AiV2,21.0,pass,0.601033
4745,2017-04-20,P1,Partial-Nucleus,NaN,AiV2,21.0,pass,0.678447
4746,2017-04-20,P1,Nucleated,NaN,AiV2,21.0,pass,NaN
4758,2017-04-20,P2,Nucleated,NaN,AiV2,21.0,pass,0.678447
4770,2017-04-21,P1,Partial-Nucleus,"Fluorescence in Pipette,Cell Dimmed",AiV2,21.0,pass,1.000000
4771,2017-04-21,P1,Nucleated,"Fluorescence in Pipette,Cell Dimmed",AiV2,21.0,pass,0.795757
4775,2017-04-21,P2,Nucleated,Fluorescence in Pipette,AiV2,21.0,pass,1.000000


In [92]:
nuc_count = nuc["Post patch?"].count()
nuc_count

1010

In [94]:
nuc_percent = float(nuc_count)/total
nuc_percent

0.5920281359906213

In [95]:
no_p1_total = ps[ps["User"] != 'P1']
no_p1_total

,Date,User,Post patch?,Notes,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,21.0,pass,0.923406
4719,2017-04-19,P2,No-Seal,NaN,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,21.0,pass,0.322631
4722,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.322631
4723,2017-04-19,P2,No-Seal,NaN,21.0,pass,0.000000
4724,2017-04-19,P2,Nucleated,NaN,21.0,pass,NaN
4727,2017-04-19,P6,No-Seal,NaN,21.0,fail,NaN
4728,2017-04-19,P6,No-Seal,NaN,21.0,pass,0.000000


In [96]:
no_p1_nuc = no_p1_total[no_p1_total["Post patch?"] == "Nucleated"]
no_p1_nuc

,Date,User,Post patch?,Notes,PCR cycles,SM_QC_PF,Resolution index
4720,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.000000
4722,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.322631
4724,2017-04-19,P2,Nucleated,NaN,21.0,pass,NaN
4758,2017-04-20,P2,Nucleated,NaN,21.0,pass,0.678447
4775,2017-04-21,P2,Nucleated,Fluorescence in Pipette,21.0,pass,1.000000
4776,2017-04-21,P2,Nucleated,Fluorescence in Pipette,21.0,pass,0.877682
4784,2017-04-21,P2,Nucleated,NaN,21.0,pass,0.556320
4807,2017-04-24,P2,Nucleated,NaN,21.0,pass,0.678447
4812,2017-04-24,P2,Nucleated,Fluorescence in Pipette One big bubble formed ...,21.0,pass,0.678447
4833,2017-04-25,P2,Nucleated,NaN,21.0,pass,1.000000


In [55]:
no_p1_count = no_p1_total["Post patch?"].count()
no_p1_count

866

In [56]:
no_p1_nuccount = no_p1_nuc["Post patch?"].count()
no_p1_nuccount

431

In [57]:
new_no_p1_percent = float(no_p1_nuccount)/no_p1_count
new_no_p1_percent

0.4976905311778291

# Calculating nucleated patch percentages by user

In [121]:
P8_total = ps[ps["User"] == 'P8']

P8_total_count = P8_total['Post patch?'].count()
P8_total_count

416

In [122]:
P8_nuc = P8_total[P8_total['Post patch?'] == 'Nucleated']
P8_nuc_count = P8_nuc['Post patch?'].count()
P8_nuc_count

211

In [123]:
float(P8_nuc_count)/P8_total_count

0.5072115384615384

In [124]:
for riguser in ['P1','P2','P6','P8']:
    user_total = ps[ps['User'] == riguser]
    user_total_count = user_total['Post patch?'].count()
    user_nuc = user_total[user_total['Post patch?'] == 'Nucleated']
    user_nuc_count = user_nuc['Post patch?'].count()
    user_percent = float(user_nuc_count)/(user_total_count)
    print riguser,': ', user_percent
    

P1 :  0.342490842491
P2 :  0.498141263941
P6 :  0.369863013699
P8 :  0.507211538462


# Obtaining nucleus dependant on internal solution version

In [21]:
standard_internal = ps[ps["Internal version"] == "AiV2"]
standard_internal = standard_internal[(standard_internal["User"] == "P2") | (standard_internal["User"] == "P8")]
standard_internal.head()
                       

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4719,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631


In [22]:
total_standard_internal = standard_internal["Post patch?"].count()
total_standard_internal

759

In [19]:
alexa_internal = ps[ps["Internal version"] == "AiV2-50uM alexa 488"]
alexa_internal = alexa_internal[(alexa_internal["User"] == "P2") | (alexa_internal["User"] == "P8")]
alexa_internal

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
6843,2017-08-29,P8,Entire-Cell,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",AiV2-50uM alexa 488,21.0,pass,0.322631
6844,2017-08-29,P8,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Swell...",AiV2-50uM alexa 488,21.0,pass,0.000000
6845,2017-08-30,P8,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Swelled No Bubble...",AiV2-50uM alexa 488,21.0,pass,0.783192
6848,2017-08-30,P8,No-Seal,NaN,NaN,"Fluorescence in Pipette,Cell Shrunk No Bubbles...",AiV2-50uM alexa 488,21.0,pass,0.000000
6850,2017-08-30,P8,No-Seal,NaN,NaN,Fluorescence in Pipette No Bubbles,AiV2-50uM alexa 488,21.0,pass,0.322631
6857,2017-08-31,P2,Nucleated,NaN,NaN,No Bubbles 0.1 units of pressure on expulsion,AiV2-50uM alexa 488,21.0,pass,1.000000
6858,2017-08-31,P2,Nucleated,NaN,NaN,Fluorescence in Pipette No Bubbles 0.1 units o...,AiV2-50uM alexa 488,21.0,pass,1.000000
6859,2017-08-31,P2,Nucleated,NaN,NaN,Fluorescence in Pipette No Bubbles 0.1 units o...,AiV2-50uM alexa 488,21.0,pass,1.000000
6862,2017-08-31,P8,Nucleated,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",AiV2-50uM alexa 488,21.0,pass,1.000000
6863,2017-08-31,P8,No-Seal,NaN,NaN,Cell Disappeared,AiV2-50uM alexa 488,21.0,pass,0.469928


In [20]:
total_alexa_internal = alexa_internal["Post patch?"].count()
total_alexa_internal

203

In [23]:
standard_internal_nuc = standard_internal[(standard_internal["Post patch?"] == "Nucleated") | (standard_internal["Post patch?"] == "Partial-Nucleus") | (standard_internal["Post patch?"] == "nucleus_present")|(standard_internal["Post patch?"] == "nucleus_visible")]
standard_internal_nuc
standard_internal_nuc_count = standard_internal_nuc["Post patch?"].count()
print "standard internal:" , float(standard_internal_nuc_count)/total_standard_internal

standard internal: 0.636363636364


In [138]:
alexa_internal_nuc = alexa_internal[(alexa_internal["Post patch?"] == "Nucleated") | (alexa_internal["Post patch?"] == "Partial-Nucleus") | (alexa_internal["Post patch?"] == "nucleus_present")|(alexa_internal["Post patch?"] == "nucleus_visible")]
alexa_internal_nuc
alexa_internal_nuc_count = alexa_internal_nuc["Post patch?"].count()
print "alexa internal:" , float(alexa_internal_nuc_count)/total_alexa_internal

alexa internal: 0.698412698413


# Rate of SMQC pass-fail by internal solution verion

In [146]:
standard_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
standard_internal_count = standard_internal["Post patch?"].count()
standard_internal_count

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1384

In [142]:
standard_internal_pass = standard_internal[standard_internal["SM_QC_PF"] == "pass"]
standard_internal_pass_count = standard_internal_pass["Post patch?"].count()
standard_internal_pass_count                                    

1079

In [145]:
print "Standard internal pass rate:" , float(standard_internal_pass_count)/standard_internal_count

Standard internal pass rate: 0.779624277457


In [147]:
alexa_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
alexa_internal_count = alexa_internal["Post patch?"].count()
alexa_internal_count

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


93

In [149]:
alexa_internal_pass = alexa_internal[alexa_internal["SM_QC_PF"] == "pass"]
alexa_internal_pass_count = alexa_internal_pass["Post patch?"].count()
alexa_internal_pass_count

76

In [150]:
print "Alexa internal pass rate:" , float(alexa_internal_pass_count)/alexa_internal_count

Alexa internal pass rate: 0.817204301075
